# 4.1  Crear modelos para cada período


Los títulos asociados a proyecto de LEY en promedio tiene 21 palabras, max 125 y mínimo 2 palabras.

Periodo_5anios : cantidad de IL sobre ley y salud
* 2019    1065
* 2014    1008
* 2009     775

Analizando proyectos de IL de Salud:
* [2009 - 2014) o [127 - 132)
    * Número total de tokens: 7652
    * Número de tokens únicos: 1713
* [2014 - 2019) o [132 - 137)
    * Número total de tokens: 9494
    * Número de tokens únicos: 1967
* [2019 - 2024) o [137 - 142)
    * Número total de tokens: 9881
    * Número de tokens únicos: 1875


**Corpus IL:**
* Las palabras aparecen pocas veces. Esto afectará el aprender del modelo. Embeddings inestables, esto afecta la detección de cambios semánticos.
  
* Esto afecta tareas posteriores
    * Analizando las frecuencia de token por período:
        * 2 veces es la mediana de veces en que aparece un token por periodo.
        * 5 veces es el promedio de veces en que aparece un token por periodo.
        * Cuartil 3 de Frecuencia = 8.5 
        * 315 palabras con frecuencias extremas

In [1]:
# Importar librerias
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import pickle
import sweetviz as sv
import sys

# Visualización
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors
from matplotlib import colormaps

from collections import Counter
from itertools import islice

from tqdm import tqdm
tqdm.pandas()

In [2]:
# Configurar path
### CONFIGURACION
load_dotenv() # Cargar las variables de entorno del archivo .env
BASE_DIR =  os.getenv("DIR_BASE")
RESULTADOS_DIR = os.getenv("DIR_DATOS_PROCESADOS") # Acceder a las variables de entorno
sys.path.append(BASE_DIR)
sys.path.append(RESULTADOS_DIR)

pd.set_option('display.max_colwidth', None)

In [ ]:
# LEER OBJETO PARA DETECTAR CAMBIOS SEMANTICOS
with open(RESULTADOS_DIR + 'periodo_5anios_df.pkl', 'rb') as file: 
    PER5anios_df = pickle.load(file)

In [4]:
PER5anios_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Periodo_5anios      3 non-null      int64 
 1   Título normalizado  3 non-null      object
 2   corpus              3 non-null      object
dtypes: int64(1), object(2)
memory usage: 204.0+ bytes


### 2- Crear modelo Word2Vec 
Modelo inestable por corpus pequeño

Entrenar embeddings Word2Vec para cada período.
Usamos Word2Vec con 

```
Word2Vec(
    sentences=sentences,
    vector_size=50,       # menor dimensión para evitar sobreajuste
    window=10,            # más grande para capturar más contexto
    min_count=2,          # más bajo para no perder vocabulario.
    workers=4,
    sg=1,                 # usar Skip-Gram, funciona mejor en corpus pequeños
    epochs=50             # entrenar más veces para compensar pocos datos
)
```

In [7]:
from gensim.models import Word2Vec
#https://radimrehurek.com/gensim/models/word2vec.html

def entrenar_word2vec(sentencias, size=50, window=10, min_count=2,  w=4, s=1, e=50, seed=1):
    return Word2Vec(sentencias, vector_size=size, window=window, min_count=min_count, workers=w, sg= s, epochs=e, seed=seed)


In [6]:
def pares_primer_paso(lista_items):
    return [(lista_items[i],lista_items[i+1]) for i in range(len(lista_items)-1)]

In [8]:
anios5_list = sorted(PER5anios_df['Periodo_5anios'].to_list())
anios5_list
anios5_pairs = pares_primer_paso(anios5_list)
anios5_pairs


[(2009, 2014), (2014, 2019)]

In [9]:
PER5anios_df['corpus'][0][:3] # 2009

[['paro',
  'cardiorrespiratorio',
  'espectaculo',
  'publico',
  'futbol',
  'regimen',
  'presupuesto',
  'minimo',
  'necesario',
  'materia',
  'prevencion',
  'atencion',
  'primaria',
  'basica'],
 ['ministerio',
  'salud',
  'nacion',
  'registro',
  'universal',
  'sanitario',
  'nacional'],
 ['incorporacion',
  'calendario',
  'nacional',
  'vacunacion',
  'dosis',
  'vacuna',
  'varicela',
  'caracter',
  'obligatorio']]

In [10]:
# Entrenar modelos de Word2Vec para cada período
i = 1 # iteración
modelos = [] # Cargamos modelos
for anios5, corpus in tqdm(zip(PER5anios_df['Periodo_5anios'], PER5anios_df['corpus'])):
    print(anios5)
    print(len(corpus))
    modelo = entrenar_word2vec(corpus, size=50, window=10, min_count=2, w=4, s=1, e=50, seed=1)
    modelos.append(modelo)
    


0it [00:00, ?it/s]

2009
775


1it [00:01,  1.74s/it]

2014
1008


2it [00:04,  2.25s/it]

2019
1065


3it [00:06,  2.27s/it]


In [11]:
## para 2009 - 2013
modelos[0].wv.most_similar("salud", topn=15)

[('poblacional', 0.6051187515258789),
 ('trabajo', 0.568244993686676),
 ('comit', 0.5647547841072083),
 ('ambiente', 0.5644502639770508),
 ('tumor', 0.5433409810066223),
 ('fortalecimiento', 0.5361788272857666),
 ('nacion', 0.5292761325836182),
 ('mistanasica', 0.5282478928565979),
 ('maligno', 0.5262343883514404),
 ('financiero', 0.52387535572052),
 ('poblacion', 0.5238097310066223),
 ('primaria', 0.5211458206176758),
 ('consultoria', 0.5189443230628967),
 ('nosocomial', 0.5157310366630554),
 ('intrahospitalaria', 0.5136682987213135)]

In [12]:
## para 2014 - 2018
modelos[1].wv.most_similar("salud", topn=15)

[('ministerio', 0.6628741025924683),
 ('nacion', 0.6424374580383301),
 ('fortalecimiento', 0.582579493522644),
 ('tumor', 0.5776768326759338),
 ('ampliar', 0.5666766166687012),
 ('central', 0.5568865537643433),
 ('nosocomial', 0.5566364526748657),
 ('odontologia', 0.5511101484298706),
 ('figura', 0.5398802161216736),
 ('intrahospitalaria', 0.5384962558746338),
 ('implantabl', 0.5323957800865173),
 ('estatal', 0.5301596522331238),
 ('fiebre', 0.513197124004364),
 ('banco', 0.5130578279495239),
 ('vida', 0.5095268487930298)]

In [13]:
## para 2019 - 2023
modelos[2].wv.most_similar("salud", topn=15)

[('figura', 0.5529130697250366),
 ('crear', 0.5481691956520081),
 ('fnpcp', 0.5183286070823669),
 ('hereditario', 0.5134453177452087),
 ('cientifica', 0.5126909613609314),
 ('bioquimica', 0.5119182467460632),
 ('modulo', 0.49890363216400146),
 ('publicacion', 0.4953594207763672),
 ('sis', 0.4892595112323761),
 ('procreacion', 0.48890772461891174),
 ('integrado', 0.4884296655654907),
 ('nacion', 0.48218780755996704),
 ('cesarea', 0.48045477271080017),
 ('ministerio', 0.4782728850841522),
 ('reproductivo', 0.47607582807540894)]

### 3- Detectar cambio semántico por Orthogonal Procrustes :   (Hamilton et al., 2016)  

Alinear embeddings con Procrustes: Alineamos los modelos secuencialmente para compararlos en el mismo espacio semántico.
* Alinean embeddings de múltiples períodos secuencialmente con Iterative Procrustes.
* Se usa el período inicial como referencia y cada nuevo período se alinea con la versión anterior.
* Esto mantiene la estabilidad en el cambio semántico y reduce el ruido en palabras con alta variabilidad.

**¿Por qué hay que alinear?**

Cuando entrenás embeddings (por ejemplo, Word2Vec) en textos de distintos años, cada modelo puede:

* Tener diferentes orientaciones en el espacio (rotaciones, reflexiones).

* Usar coordenadas diferentes para significados similares.

Aunque "salud" tenga el mismo significado en 2009 y en 2014, los vectores pueden estar en posiciones distintas.
La alineación con Procrustes soluciona esto rotando y escalando los vectores para que estén en el mismo sistema de referencia.

No cambiamos el significado de las palabras, sino que ajustamos el sistema de coordenadas para que los vectores sean comparables.

In [ ]:
from src.nlp import estabilidad_procrustes as proc
import numpy as np
from scipy.linalg import orthogonal_procrustes

In [16]:

# Función para alinear embeddings con Procrustes scipy
def alinear_procrustes(base_model, target_model):
    
    common_words = list(set(base_model.wv.index_to_key) & set(target_model.wv.index_to_key))
    #print(common_words)
    
    base_matrix = np.array([base_model.wv[word] for word in common_words])
    target_matrix = np.array([target_model.wv[word] for word in common_words])
    
    R, _ = orthogonal_procrustes(target_matrix, base_matrix)
    
    aligned_target = {word: target_model.wv[word] @ R for word in common_words}
    return aligned_target



In [17]:
vocab_comun = ['salud','accion']
X = np.array([modelos[0].wv[word] for word in vocab_comun])
Y = np.array([modelos[1].wv[word] for word in vocab_comun])

In [ ]:
X[0]

array([-0.76661927, -0.4325066 ,  0.25832516,  0.38662606, -0.05662229,
       -0.64894587, -0.24123998,  0.02558997, -0.21201323, -0.47661552,
       -0.40251607, -0.23828717,  0.4128283 ,  0.27515277, -0.34597382,
        0.24998742,  0.29582503, -0.29708284, -0.15107588, -0.7316394 ,
        0.0605019 , -0.40928772,  0.4454853 , -0.5765498 ,  0.55589855,
       -0.10620517, -0.27302548,  0.06232678, -0.23466651,  0.68941194,
        0.871782  , -0.1426986 ,  0.13816154, -0.1967091 , -0.05106308,
       -0.41106868,  0.84974825,  0.3773513 ,  0.04463656, -0.3876453 ,
        0.22333625,  0.21231668, -0.19567092,  0.18744278,  0.7696105 ,
        0.33415315,  0.63159174,  0.557659  , -0.12445208, -0.20140213],
      dtype=float32)

In [ ]:
Y[0] # el que voy a alinear

array([ 0.079476  , -0.55637676,  0.46066153,  0.5598775 ,  0.48113063,
        0.3931902 ,  0.42459077,  1.630586  , -0.4863917 ,  0.3236691 ,
       -0.5581608 ,  0.12543051, -0.1841846 , -0.2655787 , -0.00236468,
        0.21027584,  0.93432343,  0.43968663, -0.23600185, -0.13143644,
       -0.5664267 , -0.24251568,  0.63501155,  0.214907  , -0.1532986 ,
        0.14523982, -0.27386513,  0.12166338,  0.08930604,  0.14271961,
       -0.50580806,  0.28047282,  0.25264218, -0.30923456, -0.09082257,
       -0.08189114,  0.35400146, -0.5037463 ,  0.63542306, -0.45921707,
        0.24461576,  0.2660912 , -0.7460097 ,  0.25073925,  0.17295976,
       -0.338223  ,  0.19380197, -0.65161324, -0.41050783, -0.24390787],
      dtype=float32)

In [ ]:
# Calcular la rotación ortogonal
R, _ = orthogonal_procrustes(Y, X)
# Aplicar la rotación a todos los embeddings del segundo modelo
modeloaligned = {word: modelos[1].wv[word] @ R for word in vocab_comun}

In [30]:
modeloaligned

{'salud': array([-0.83348626, -0.47276273,  0.28691936,  0.41803658, -0.06466002,
        -0.71627843, -0.26220977,  0.03015995, -0.23308145, -0.5224773 ,
        -0.43944013, -0.26498482,  0.45261234,  0.3049677 , -0.38380143,
         0.27107888,  0.3247075 , -0.32217035, -0.16783506, -0.80108595,
         0.06493317, -0.44274977,  0.49099347, -0.63768566,  0.60915005,
        -0.11684465, -0.2984457 ,  0.06832987, -0.2624092 ,  0.75502205,
         0.9536005 , -0.15109786,  0.14496748, -0.21565463, -0.0626744 ,
        -0.44765797,  0.9315549 ,  0.41567862,  0.05258478, -0.42501223,
         0.2477837 ,  0.22664644, -0.20954148,  0.19866678,  0.8449272 ,
         0.36518753,  0.6902768 ,  0.61090595, -0.13362516, -0.21758482],
       dtype=float32),
 'accion': array([ 0.43567222, -0.06188819,  0.5899899 , -0.5007597 , -0.3444731 ,
        -0.93525624,  0.14581755,  0.26963255, -0.19255148, -0.2505065 ,
         0.00810702, -0.5833283 ,  0.22430229,  0.5504106 , -0.73323756,
        

In [ ]:
# Usando otro proyect ort
modelo_alin = proc.smart_procrustes_align_gensim(modelos[0], modelos[1])
modelo_alin.wv['salud']

array([-0.5901762 , -0.32099983,  0.1798446 , -0.12335356, -0.08840335,
       -0.2636055 , -0.15025623,  0.380592  , -0.23493706,  0.00736756,
       -0.33755237, -0.23405166,  0.4462552 ,  0.20498578, -0.38673124,
        0.20506981,  0.938567  , -0.45597395, -0.15547249, -1.2914217 ,
        0.19843052, -0.48357117,  0.33145693, -0.23080048,  0.16478555,
        0.25518608, -0.15124999, -0.05656921, -0.31502035,  0.9164427 ,
        0.4544109 ,  0.01978845, -0.62647617,  0.38316867, -0.2646639 ,
       -0.04165461,  1.0169672 ,  0.11497834,  0.25669977, -0.6127046 ,
        0.14500146,  0.49579167, -0.51072884, -0.10588026,  0.8235962 ,
       -0.00986712,  0.761114  ,  0.54713416, -0.06644504, -0.14222562],
      dtype=float32)

### 4- Detectar cambio semántico por NN - Nearest Neighbors:  (Gonen et al., 2020)
Introducen la intersección @k, es decir, la intersección de los k vecinos más cercanos de cada palabra en cada corpus, para medir la diferencia entre palabras vecinas. 

📌 Idea clave: En lugar de alinear embeddings, compara directamente los vecinos más cercanos en el espacio semántico.
* Se calcula la intersección de los k vecinos más cercanos de una palabra en diferentes períodos.
* Cuanto más cambien los vecinos, mayor será el cambio semántico.

In [34]:
from src.nlp import estabilidad_NN as nn

In [ ]:
def palabras_elegibles(m1, m2, palabras_top_m1, palabras_menosFrec_m1,
                   palabras_top_m2, palabras_menosFrec_m2):
    ''' 
    Recopila palabras aptas para el cálculo del desplazamiento semántico a partir de la intersección de los vocabularios que cumplen umbrales de frecuencia específicos
    '''
    
    m1_vocab = [key for key, value in m1.wv.key_to_index.items() if key != ' ']
    m2_vocab = [key for key, value in m2.wv.key_to_index.items() if key != ' ']

    interseccion = set(m1_vocab).intersection(set(m2_vocab))
    # Palabras muy frecuentes - top
    top_frec = set(palabras_top_m1 + palabras_top_m2)
    # Palabras menos frecuentes en base a umbral, ejemplo media
    menos_frec = set(palabras_menosFrec_m1 + palabras_menosFrec_m2)

    # Palabras limpias para buscar cambios de uso
    final_list = [w for w in interseccion if
                  w not in top_frec and w not in menos_frec and w != ' ']
   
    print("Cantidad Final lista de palabras: ",len(final_list))

    return m1_vocab, m2_vocab, final_list

def vecinos_elegibles(vocab1, vocab2, menos_frec_union):
    ''' 
    Recopila palabras que son vecinos elegibles de las palabras estudiadas para el cambio semántico.
    Las vecinos elegibles deben estar en ambos vocabularios modelo y 
    deben aparecer más de determinadas veces en cada corpus, para ello se tiene en cuenta los menos frecuentes.
    '''

    interseccion_vocabs = list(set(vocab1) & set(vocab2))
    # se considera palabras repetidas en general
    vecinos_plausibles = [w for w in interseccion_vocabs if
                           w not in menos_frec_union and w != ' ']

    return vecinos_plausibles

def recolectar_vecinos_elegibles(palabra, m, vecinos_plausibles, topn_vecinos):
    c = 0
    out = []
    for w, s in m.wv.most_similar(positive=[palabra], topn=topn_vecinos):
        if w in vecinos_plausibles:
            out.append(w)
            c += 1
        if c == topn_vecinos:
            break

    return (out)

In [36]:
# Umbrales cables
umbral_top = 5
umbral_frec_menos = 5
umbral_frec_menos_full  = 9 
topn_vecinos = 100

In [ ]:
#Recopilar palabras para el análisis de cambio semántico que cumplan con los umbrales
#Contar la frecuencia de palabras por década
frec_anio_dic = {}
top_palabras_dic = {}
palabras_menosFrec_dic = {}
palabras_menosFrecFull_dic = {}

for anio in anios5_list:
        df = pd.read_csv(RESULTADOS_DIR+'/archivos_out/frec_para_datos_limpios_por_desplaz_semantico_anios5_'+str(anio)+'.csv')
        print(anio)

        print(df.Frecuencia.describe().apply(lambda x: format(x, 'f')))
        df = df.sort_values('Frecuencia', ascending=False)
        frec_anio_dic[anio] = df

        top_palabras_dic[anio] = df.Palabra.head(umbral_top).to_list()
        palabras_menosFrec_dic[anio] = df.loc[df.Frecuencia < umbral_frec_menos].Palabra.to_list()
        palabras_menosFrecFull_dic[anio] = df.loc[df.Frecuencia < umbral_frec_menos_full].Palabra.to_list()


2009
count    1713.000000
mean        4.467017
std        10.621856
min         1.000000
25%         1.000000
50%         2.000000
75%         3.000000
max       169.000000
Name: Frecuencia, dtype: object
2014
count    1967.000000
mean        4.826640
std        12.238140
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max       203.000000
Name: Frecuencia, dtype: object
2019
count    1875.000000
mean        5.269867
std        13.972901
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max       256.000000
Name: Frecuencia, dtype: object


In [ ]:
m1 = modelos[0]
m2 = modelos[1]
m1_vocab, m2_vocab, final_list = palabras_elegibles(m1, m2, 
                top_palabras_dic[2009], palabras_menosFrec_dic[2009],
                top_palabras_dic[2014], palabras_menosFrec_dic[2014]
                )
palabras_menos_union = set(palabras_menosFrecFull_dic[2009]+palabras_menosFrecFull_dic[2014])
vecinos = vecinos_elegibles(m1_vocab, m2_vocab, palabras_menos_union)

Cantidad Final lista de palabras:  246


In [41]:
palabra = "salud"
neighbors_t1 = recolectar_vecinos_elegibles(palabra, m1, vecinos, topn_vecinos)
neighbors_t2 = recolectar_vecinos_elegibles(palabra, m2, vecinos, topn_vecinos)
neighbors = set(neighbors_t1).intersection(set(neighbors_t2)) 
score = -len(neighbors)
print(neighbors_t1)
print(neighbors_t2)
print(neighbors)

['nacion', 'ministerio', 'atencion', 'registro', 'cancer', 'cuidado', 'derecho', 'federal', 'ley', 'publicar', 'acceso', 'beneficiario']
['ministerio', 'nacion', 'cuidado', 'materno', 'creacion', 'registro', 'fondo', 'nutricion', 'atencion', 'medico', 'persona']
{'nacion', 'cuidado', 'ministerio', 'atencion', 'registro'}
